In [1]:
import mlflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
mlflow.search_experiments(filter_string='name LIKE "% CWRU"')

[<Experiment: artifact_location='file:///NIA_agri/home/akdakd55/sparse-bfd/mlruns/902254403804568773', creation_time=1696486412582, experiment_id='902254403804568773', last_update_time=1696486412582, lifecycle_stage='active', name='Compound Pruning WDCNN CWRU', tags={}>,
 <Experiment: artifact_location='file:///NIA_agri/home/akdakd55/sparse-bfd/mlruns/622143010203640013', creation_time=1695731517709, experiment_id='622143010203640013', last_update_time=1695731517709, lifecycle_stage='active', name='STFTCNN Pruning CWRU', tags={}>,
 <Experiment: artifact_location='file:///NIA_agri/home/akdakd55/sparse-bfd/mlruns/347095940408501557', creation_time=1695176868466, experiment_id='347095940408501557', last_update_time=1695176868466, lifecycle_stage='active', name='STIM-CNN No Dropout Pruning CWRU', tags={}>,
 <Experiment: artifact_location='file:///NIA_agri/home/akdakd55/sparse-bfd/mlruns/911293835925152270', creation_time=1695045741038, experiment_id='911293835925152270', last_update_time=1

In [8]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'Featuremap Pruning %'")
runs = mlflow.search_runs(filter_string="params.model_name = 'wdcnn' and params.datapath = 'data/mfpt'", experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_mfpt{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_mfpt']

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(30)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_mfpt-4  \
params.epochs params.activation_sparsity                           
100           0.3                                  80.62 (±1.26)   
              0.5                                  81.31 (±0.92)   
              0.7                                  80.25 (±0.73)   
              0.9                                  76.13 (±0.88)   
              None                                 68.08 (±2.16)   
200           0.3                                  80.25 (±1.07)   
              0.5                                  82.06 (±0.97)   
              0.7                                  79.57 (±1.06)   
              0.9                                  75.51 (±1.04)   
              None                                 67.78 (±1.96)   
300           0.3                                  81.19 (±1.25)   
              0.5                                  81.85 (±0.82)   
              0.7                                  79.47 (±1.08)   
              0.9                                  74.09 (±1.29)   
              None                                 69.32 (±2.02)   

                                         metrics.test/acc_mfpt-2  \
params.epochs params.activation_sparsity                           
100           0.3                                  90.07 (±0.90)   
              0.5                                  89.75 (±0.73)   
              0.7                                  88.57 (±0.54)   
              0.9                                  84.01 (±0.85)   
              None                                 76.39 (±2.00)   
200           0.3                                  90.21 (±0.69)   
              0.5                                  90.50 (±0.63)   
              0.7                                  88.24 (±0.80)   
              0.9                                  83.79 (±1.06)   
              None                                 76.55 (±1.41)   
300           0.3                                  91.05 (±0.88)   
              0.5                                  90.33 (±0.60)   
              0.7                                  88.19 (±0.86)   
              0.9                                  81.98 (±1.10)   
              None                                 76.60 (±1.72)   

                                         metrics.test/acc_mfpt0  \
params.epochs params.activation_sparsity                          
100           0.3                                 95.79 (±0.39)   
              0.5                                 94.72 (±0.43)   
              0.7                                 93.00 (±0.41)   
              0.9                                 89.27 (±0.53)   
              None                                84.37 (±1.82)   
200           0.3                                 96.25 (±0.31)   
              0.5                                 95.38 (±0.37)   
              0.7                                 93.36 (±0.58)   
              0.9                                 89.30 (±0.76)   
              None                                85.65 (±1.47)   
300           0.3                                 96.80 (±0.42)   
              0.5                                 95.74 (±0.38)   
              0.7                                 93.54 (±0.53)   
              0.9                                 88.77 (±0.85)   
              None                                84.38 (±1.60)   

                                         metrics.test/acc_mfpt2  \
params.epochs params.activation_sparsity                          
100           0.3                                 98.02 (±0.26)   
              0.5                                 97.20 (±0.30)   
              0.7                                 95.69 (±0.28)   
              0.9                                 92.48 (±0.51)   
              None                                92.32 (±1.53)   
200           0.3                                 98.45 (±0.19)   
              0.5                                 9

In [ ]:
epochs = 300

fig, axes = plt.subplots(1, 2, sharey='row', figsize=(20, 10))

width = 0.35
labels = [str(db) for db in range(-4, 6, 2)] + ['noise-free']
x = np.arange(len(labels))

for environ, ax in zip(environs, axes):
    unstr_bar = ax.bar(x - width/2, round(means.loc[str(epochs), 'activation'][f'metrics.test/acc_{environ}'], 2), width, label='Unstructured', color='C1', linewidth=1.5, edgecolor='black')
    str_bar = ax.bar(x + width/2, round(means.loc[str(epochs), 'channel'][f'metrics.test/acc_{environ}'], 2), width, label='Structured', color='C2', linewidth=1.5, edgecolor='black')

    ax.bar_label(unstr_bar, padding=3, fontsize=18)
    ax.bar_label(str_bar, padding=3, fontsize=18)

    dense = dense_acc.loc[str(epochs)][f'metrics.test/acc_{environ}']

    dense_line = ax.hlines(dense, 0, 1, transform=ax.get_yaxis_transform(), linewidth=3, linestyles='dashed', color='C0')

    if environ == 'cwru':
        ax.set_title('Noise-free', fontsize=28)
    else:
        ax.set_title('Noisy (0dB)', fontsize=28)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.tick_params(axis='both', labelsize=24, which='major', bottom=True, left=True, grid_color='none', color='black', length=8, width=2)

axes[0].set_ylim(30)

ax = fig.add_subplot(111, frameon=False)
ax.set_xlabel('Sparsity', fontsize=28, labelpad=20)
ax.set_ylabel('Accuracy (%)', fontsize=28, labelpad=25)
ax.tick_params(grid_color='none',labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
ax.set_title(f'Epoch : {epochs}', y=1.1, fontsize=32)

legend = fig.legend([unstr_bar, str_bar, dense_line], ('Unstructured', 'Structured', 'Dense'), loc='lower left', bbox_to_anchor=(.99, 0.45), fontsize=24)
legend.get_frame().set_linewidth(2)
legend.get_frame().set_edgecolor('black')
fig.tight_layout()

In [9]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'Featuremap Pruning CWRU'")
runs = mlflow.search_runs(filter_string="params.model_name = 'stimcnn' and params.activation_drop = 'featuremap'", experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_cwru{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_cwru']

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(30)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_cwru-4  \
params.epochs params.activation_sparsity                           
100           0.3                                   9.81 (±1.40)   
              0.5                                   9.31 (±0.97)   
              0.7                                   8.93 (±0.83)   
              0.9                                  10.76 (±1.95)   
200           0.3                                   8.86 (±1.27)   
              0.5                                   8.11 (±0.40)   
              0.7                                   8.31 (±0.60)   
              0.9                                   9.31 (±1.00)   
300           0.3                                   8.67 (±0.93)   
              0.5                                   7.80 (±0.02)   
              0.7                                   8.10 (±0.32)   
              0.9                                   8.68 (±0.67)   

                                         metrics.test/acc_cwru-2  \
params.epochs params.activation_sparsity                           
100           0.3                                  13.35 (±3.13)   
              0.5                                  12.22 (±2.21)   
              0.7                                  10.82 (±1.67)   
              0.9                                  12.82 (±2.71)   
200           0.3                                  11.32 (±2.25)   
              0.5                                   9.31 (±1.34)   
              0.7                                   9.39 (±1.58)   
              0.9                                  11.31 (±1.66)   
300           0.3                                  11.07 (±2.68)   
              0.5                                   8.10 (±0.25)   
              0.7                                   9.29 (±1.25)   
              0.9                                  10.32 (±1.51)   

                                         metrics.test/acc_cwru0  \
params.epochs params.activation_sparsity                          
100           0.3                                 19.73 (±4.98)   
              0.5                                 18.26 (±4.14)   
              0.7                                 15.76 (±3.23)   
              0.9                                 16.87 (±4.03)   
200           0.3                                 18.02 (±3.54)   
              0.5                                 12.60 (±2.66)   
              0.7                                 12.12 (±2.79)   
              0.9                                 15.63 (±2.92)   
300           0.3                                 15.63 (±3.92)   
              0.5                                 10.08 (±1.32)   
              0.7                                 12.18 (±2.98)   
              0.9                                 13.48 (±2.62)   

                                         metrics.test/acc_cwru2  \
params.epochs params.activation_sparsity                          
100           0.3                                 30.17 (±6.24)   
              0.5                                 28.29 (±6.18)   
              0.7                                 23.91 (±4.77)   
              0.9                                 24.29 (±5.83)   
200           0.3                                 30.06 (±4.41)   
              0.5                                 20.31 (±4.11)   
              0.7                                 18.17 (±4.13)   
              0.9                                 23.72 (±4.59)   
300           0.3                                 26.23 (±5.02)   
              0.5                                 16.38 (±3.40)   
              0.7                                 17.54 (±4.42)   
              0.9                                 20.24 (±4.17)   

                                         metrics.test/acc_cwru4  \
params.epochs params.activation_sparsity                          
100           0.3                                 46.88 (±6.97)   
              0.5                                 42.29 

In [10]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'Featuremap Pruning CWRU'")
runs = mlflow.search_runs(filter_string="params.model_name = 'stimcnn' and params.activation_drop = 'none'", experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_cwru{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_cwru']

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(30)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

,,metrics.test/acc_cwru-4,metrics.test/acc_cwru-2,metrics.test/acc_cwru0,metrics.test/acc_cwru2,metrics.test/acc_cwru4,metrics.test/acc_cwru
params.epochs,params.activation_sparsity,,,,,,
100,None,14.50 (±4.09),23.60 (±5.27),34.53 (±6.23),46.87 (±5.00),58.90 (±3.37),96.99 (±4.25)
200,None,10.51 (±2.39),14.78 (±4.80),22.13 (±5.98),35.61 (±6.22),50.88 (±4.04),99.52 (±0.41)
300,None,7.99 (±0.24),10.47 (±2.05),18.10 (±4.89),30.78 (±5.71),44.85 (±5.28),99.54 (±0.27)


In [71]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'STIM-CNN %'")
runs = mlflow.search_runs(filter_string="params.model_name = 'stimcnn' and params.activation_drop != 'featuremap'", experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_cwru{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_cwru']

n = len(runs['params.seed'].unique())

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(n)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_cwru-4  \
params.epochs params.activation_sparsity                           
100           0.3                                  22.09 (±4.38)   
              0.5                                  19.00 (±4.75)   
              0.7                                  14.74 (±3.17)   
              0.9                                  15.80 (±2.56)   
200           0.3                                  18.53 (±4.57)   
              0.5                                  17.92 (±4.43)   
              0.7                                  11.79 (±1.39)   
              0.9                                  12.19 (±1.33)   
300           0.3                                  16.44 (±4.02)   
              0.5                                  14.41 (±4.03)   
              0.7                                  11.16 (±1.70)   
              0.9                                  12.50 (±1.66)   

                                         metrics.test/acc_cwru-2  \
params.epochs params.activation_sparsity                           
100           0.3                                  31.67 (±4.71)   
              0.5                                  26.02 (±5.49)   
              0.7                                  22.01 (±4.64)   
              0.9                                  24.37 (±3.91)   
200           0.3                                  26.09 (±5.56)   
              0.5                                  24.36 (±5.12)   
              0.7                                  18.57 (±3.72)   
              0.9                                  17.18 (±2.81)   
300           0.3                                  22.96 (±5.21)   
              0.5                                  19.51 (±4.65)   
              0.7                                  14.33 (±2.75)   
              0.9                                  18.43 (±3.39)   

                                         metrics.test/acc_cwru0  \
params.epochs params.activation_sparsity                          
100           0.3                                 40.86 (±4.92)   
              0.5                                 36.95 (±5.50)   
              0.7                                 31.86 (±5.60)   
              0.9                                 37.67 (±5.49)   
200           0.3                                 34.82 (±5.71)   
              0.5                                 32.74 (±5.64)   
              0.7                                 26.92 (±4.89)   
              0.9                                 28.34 (±5.28)   
300           0.3                                 29.77 (±5.77)   
              0.5                                 26.38 (±5.42)   
              0.7                                 19.71 (±3.82)   
              0.9                                 28.00 (±5.05)   

                                         metrics.test/acc_cwru2  \
params.epochs params.activation_sparsity                          
100           0.3                                 48.57 (±5.11)   
              0.5                                 48.28 (±5.33)   
              0.7                                 43.96 (±6.22)   
              0.9                                 51.76 (±6.11)   
200           0.3                                 44.68 (±5.12)   
              0.5                                 42.52 (±5.92)   
              0.7                                 39.36 (±5.22)   
              0.9                                 43.68 (±6.64)   
300           0.3                                 38.63 (±5.39)   
              0.5                                 35.04 (±5.98)   
              0.7                                 29.25 (±5.71)   
              0.9                                 41.29 (±5.77)   

                                         metrics.test/acc_cwru4  \
params.epochs params.activation_sparsity                          
100           0.3                                 59.13 (±5.04)   
              0.5                                 61.90 

In [7]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'STIM-CNN %'")
runs = mlflow.search_runs(filter_string="params.model_name = 'stimcnn'", experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_cwru{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_cwru']

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(30)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_cwru-4  \
params.epochs params.activation_sparsity                           
100           0.3                                   9.90 (±1.23)   
              0.5                                  10.42 (±1.31)   
              0.7                                  10.04 (±1.20)   
              0.9                                  10.20 (±1.52)   
200           0.3                                   8.91 (±0.60)   
              0.5                                   8.76 (±0.56)   
              0.7                                   8.60 (±1.13)   
              0.9                                  10.30 (±1.31)   
300           0.5                                     nan (±nan)   
              0.7                                     nan (±nan)   
              0.9                                     nan (±nan)   

                                         metrics.test/acc_cwru-2  \
params.epochs params.activation_sparsity                           
100           0.3                                  13.61 (±2.87)   
              0.5                                  14.50 (±2.65)   
              0.7                                  13.49 (±2.42)   
              0.9                                  12.24 (±2.30)   
200           0.3                                  11.44 (±1.47)   
              0.5                                  10.63 (±1.30)   
              0.7                                   9.56 (±1.93)   
              0.9                                  11.75 (±1.55)   
300           0.5                                     nan (±nan)   
              0.7                                     nan (±nan)   
              0.9                                     nan (±nan)   

                                         metrics.test/acc_cwru0  \
params.epochs params.activation_sparsity                          
100           0.3                                 20.04 (±4.55)   
              0.5                                 21.96 (±4.50)   
              0.7                                 19.87 (±4.12)   
              0.9                                 17.06 (±3.60)   
200           0.3                                 18.03 (±3.10)   
              0.5                                 15.77 (±3.20)   
              0.7                                 11.65 (±2.98)   
              0.9                                 16.13 (±2.64)   
300           0.5                                    nan (±nan)   
              0.7                                    nan (±nan)   
              0.9                                    nan (±nan)   

                                         metrics.test/acc_cwru2  \
params.epochs params.activation_sparsity                          
100           0.3                                 30.88 (±6.11)   
              0.5                                 33.29 (±6.67)   
              0.7                                 29.90 (±6.22)   
              0.9                                 25.81 (±5.50)   
200           0.3                                 28.87 (±5.00)   
              0.5                                 25.82 (±5.53)   
              0.7                                 16.39 (±4.18)   
              0.9                                 23.52 (±3.81)   
300           0.5                                    nan (±nan)   
              0.7                                    nan (±nan)   
              0.9                                    nan (±nan)   

                                         metrics.test/acc_cwru4  \
params.epochs params.activation_sparsity                          
100           0.3                                 46.29 (±6.55)   
              0.5                                 47.17 (±8.26)   
              0.7                                 43.52 (±7.85)   
              0.9                                 39.41 (±7.37)   
200           0.3                                 43.17 (±6.36)   
              0.5                                 41.06 (±

In [10]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'Featuremap Pruning %'")
runs = mlflow.search_runs(filter_string="params.datapath = 'data/ottawa'",experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_ottawa{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_ottawa']


runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(30)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_ottawa-4  \
params.epochs params.activation_sparsity                             
100           0.3                                    57.49 (±1.70)   
              0.5                                    56.12 (±1.95)   
              0.7                                    50.94 (±2.33)   
              0.9                                    35.69 (±3.10)   
              None                                   50.06 (±3.21)   
200           0.3                                    56.98 (±1.81)   
              0.5                                    58.12 (±2.10)   
              0.7                                    51.04 (±3.15)   
              0.9                                    35.20 (±2.85)   
              None                                   46.56 (±3.02)   
300           0.3                                    57.14 (±1.81)   
              0.5                                    57.73 (±2.44)   
              0.7                                    51.27 (±3.17)   
              0.9                                    35.65 (±2.85)   
              None                                   45.06 (±2.02)   

                                         metrics.test/acc_ottawa-2  \
params.epochs params.activation_sparsity                             
100           0.3                                    68.44 (±1.73)   
              0.5                                    69.92 (±2.30)   
              0.7                                    64.18 (±2.68)   
              0.9                                    44.28 (±3.78)   
              None                                   59.10 (±3.08)   
200           0.3                                    67.24 (±1.93)   
              0.5                                    70.83 (±2.30)   
              0.7                                    62.88 (±3.74)   
              0.9                                    43.31 (±3.58)   
              None                                   56.22 (±2.98)   
300           0.3                                    67.15 (±1.90)   
              0.5                                    70.70 (±2.49)   
              0.7                                    63.14 (±3.65)   
              0.9                                    43.85 (±3.51)   
              None                                   54.48 (±2.33)   

                                         metrics.test/acc_ottawa0  \
params.epochs params.activation_sparsity                            
100           0.3                                   78.18 (±1.67)   
              0.5                                   81.93 (±2.41)   
              0.7                                   76.68 (±2.37)   
              0.9                                   53.83 (±4.06)   
              None                                  67.71 (±2.90)   
200           0.3                                   76.35 (±2.06)   
              0.5                                   82.05 (±2.33)   
              0.7                                   74.23 (±3.48)   
              0.9                                   52.43 (±4.00)   
              None                                  65.82 (±2.57)   
300           0.3                                   75.94 (±2.00)   
              0.5                                   82.00 (±2.18)   
              0.7                                   74.62 (±3.40)   
              0.9                                   53.15 (±3.80)   
              None                                  64.22 (±2.21)   

                                         metrics.test/acc_ottawa2  \
params.epochs params.activation_sparsity                            
100           0.3                                   86.51 (±1.37)   
              0.5                                   89.90 (±1.98)   
              0.7                                   85.59 (±1.57)   
              0.9                                   62.62 (±3.74)   
              None                                  75.44 (±2.73)   
20

In [39]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'STFTCNN %'")
runs = mlflow.search_runs(filter_string="params.model_name = 'stftcnn' and params.activation_drop != 'activation'",experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_cwru{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_cwru']

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(30)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_cwru-4  \
params.epochs params.activation_sparsity                           
100           0.3                                  38.33 (±4.70)   
              None                                 50.59 (±4.03)   
200           0.3                                  41.80 (±5.22)   
              None                                 51.83 (±4.97)   
300           0.3                                  45.20 (±5.60)   
              None                                 55.34 (±2.81)   

                                         metrics.test/acc_cwru-2  \
params.epochs params.activation_sparsity                           
100           0.3                                  46.84 (±4.09)   
              None                                 58.09 (±3.72)   
200           0.3                                  49.88 (±4.77)   
              None                                 60.64 (±4.67)   
300           0.3                                  55.16 (±5.58)   
              None                                 65.05 (±2.84)   

                                         metrics.test/acc_cwru0  \
params.epochs params.activation_sparsity                          
100           0.3                                 55.03 (±3.99)   
              None                                65.46 (±3.88)   
200           0.3                                 57.83 (±4.57)   
              None                                69.22 (±4.14)   
300           0.3                                 63.28 (±5.66)   
              None                                74.16 (±2.65)   

                                         metrics.test/acc_cwru2  \
params.epochs params.activation_sparsity                          
100           0.3                                 62.93 (±4.07)   
              None                                72.05 (±4.19)   
200           0.3                                 66.40 (±4.30)   
              None                                77.25 (±3.88)   
300           0.3                                 71.51 (±5.52)   
              None                                82.08 (±2.49)   

                                         metrics.test/acc_cwru4  \
params.epochs params.activation_sparsity                          
100           0.3                                 71.42 (±3.64)   
              None                                78.89 (±4.40)   
200           0.3                                 74.86 (±4.03)   
              None                                84.18 (±3.62)   
300           0.3                                 79.34 (±4.97)   
              None                                88.09 (±2.17)   

                                         metrics.test/acc_cwru  
params.epochs params.activation_sparsity                        
100           0.3                                99.67 (±0.05)  
              None                               99.83 (±0.06)  
200           0.3                                99.71 (±0.05)  
              None                               99.83 (±0.04)  
300           0.3                                99.77 (±0.08)  
              None                               99.77 (±0.12)

In [3]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'STFTCNN %'")
runs = mlflow.search_runs(filter_string="params.model_name = 'stftcnn' and params.activation_drop != 'featuremap'",experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_cwru{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_cwru']

n = len(runs['params.seed'].unique())

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(n)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_cwru-4  \
params.epochs params.activation_sparsity                           
100           0.3                                  57.14 (±2.47)   
              0.5                                  59.59 (±1.42)   
              None                                 50.59 (±4.03)   
200           0.3                                  59.10 (±2.76)   
              0.5                                  58.34 (±2.37)   
              None                                 51.83 (±4.97)   
300           0.3                                  59.66 (±2.88)   
              0.5                                   42.34 (±nan)   
              None                                 55.34 (±2.81)   

                                         metrics.test/acc_cwru-2  \
params.epochs params.activation_sparsity                           
100           0.3                                  65.91 (±2.61)   
              0.5                                  68.32 (±1.78)   
              None                                 58.09 (±3.72)   
200           0.3                                  67.48 (±2.94)   
              0.5                                  66.96 (±2.65)   
              None                                 60.64 (±4.67)   
300           0.3                                  69.28 (±3.00)   
              0.5                                   51.45 (±nan)   
              None                                 65.05 (±2.84)   

                                         metrics.test/acc_cwru0  \
params.epochs params.activation_sparsity                          
100           0.3                                 74.42 (±2.43)   
              0.5                                 77.50 (±1.97)   
              None                                65.46 (±3.88)   
200           0.3                                 75.72 (±3.14)   
              0.5                                 76.23 (±2.57)   
              None                                69.22 (±4.14)   
300           0.3                                 78.11 (±2.93)   
              0.5                                  60.77 (±nan)   
              None                                74.16 (±2.65)   

                                         metrics.test/acc_cwru2  \
params.epochs params.activation_sparsity                          
100           0.3                                 81.99 (±2.23)   
              0.5                                 85.82 (±1.69)   
              None                                72.05 (±4.19)   
200           0.3                                 83.17 (±2.72)   
              0.5                                 85.08 (±2.21)   
              None                                77.25 (±3.88)   
300           0.3                                 86.13 (±2.51)   
              0.5                                  70.85 (±nan)   
              None                                82.08 (±2.49)   

                                         metrics.test/acc_cwru4  \
params.epochs params.activation_sparsity                          
100           0.3                                 88.96 (±1.89)   
              0.5                                 92.38 (±1.23)   
              None                                78.89 (±4.40)   
200           0.3                                 89.90 (±2.10)   
              0.5                                 92.06 (±1.44)   
              None                                84.18 (±3.62)   
300           0.3                                 92.00 (±1.75)   
              0.5                                  82.74 (±nan)   
              None                                88.09 (±2.17)   

                                         metrics.test/acc_cwru  
params.epochs params.activation_sparsity                        
100           0.3                                99.76 (±0.06)  
              0.5                                99.49 (±0.07)  
              None                               99.83 (±0.06)  
200 

In [75]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'Featuremap Pruning CWRU'")
runs = mlflow.search_runs(filter_string="params.model_name = 'ticnn'",experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_cwru{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_cwru']

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(30)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_cwru-4  \
params.epochs params.activation_sparsity                           
100           0.3                                  79.24 (±1.70)   
              None                                 74.84 (±1.57)   
200           0.3                                  79.32 (±2.02)   
              None                                 77.64 (±1.20)   
300           0.3                                  82.38 (±1.33)   
              None                                 78.81 (±1.73)   

                                         metrics.test/acc_cwru-2  \
params.epochs params.activation_sparsity                           
100           0.3                                  90.98 (±0.92)   
              None                                 85.39 (±1.19)   
200           0.3                                  91.90 (±0.90)   
              None                                 87.10 (±1.03)   
300           0.3                                  93.36 (±0.90)   
              None                                 88.78 (±1.08)   

                                         metrics.test/acc_cwru0  \
params.epochs params.activation_sparsity                          
100           0.3                                 96.29 (±0.46)   
              None                                91.55 (±0.77)   
200           0.3                                 96.80 (±0.42)   
              None                                92.97 (±0.73)   
300           0.3                                 97.15 (±0.41)   
              None                                93.79 (±0.67)   

                                         metrics.test/acc_cwru2  \
params.epochs params.activation_sparsity                          
100           0.3                                 97.94 (±0.28)   
              None                                96.09 (±0.61)   
200           0.3                                 98.18 (±0.29)   
              None                                97.17 (±0.53)   
300           0.3                                 98.34 (±0.28)   
              None                                97.38 (±0.56)   

                                         metrics.test/acc_cwru4  \
params.epochs params.activation_sparsity                          
100           0.3                                 98.46 (±0.25)   
              None                                98.77 (±0.32)   
200           0.3                                 98.75 (±0.26)   
              None                                99.20 (±0.25)   
300           0.3                                 98.87 (±0.16)   
              None                                98.94 (±0.43)   

                                         metrics.test/acc_cwru  
params.epochs params.activation_sparsity                        
100           0.3                                98.98 (±0.19)  
              None                               99.30 (±0.32)  
200           0.3                                99.31 (±0.25)  
              None                               99.52 (±0.20)  
300           0.3                                99.41 (±0.11)  
              None                               99.42 (±0.36)

In [30]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'Featuremap Pruning MFPT'")
runs = mlflow.search_runs(filter_string="params.model_name = 'ticnn'",experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_mfpt{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_mfpt']

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(30)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_mfpt-4  \
params.epochs params.activation_sparsity                           
100           0.3                                  91.96 (±1.12)   
              0.5                                  90.78 (±1.29)   
              0.7                                  88.63 (±1.27)   
              0.9                                  87.40 (±1.08)   
              None                                 79.79 (±1.11)   
200           0.3                                  91.29 (±1.76)   
              0.5                                  92.25 (±1.38)   
              0.7                                  86.69 (±2.21)   
              0.9                                  85.21 (±1.89)   
              None                                 79.72 (±1.47)   
300           0.3                                  89.71 (±1.55)   
              0.5                                  89.48 (±1.59)   
              0.7                                  86.61 (±2.23)   
              0.9                                  82.93 (±1.68)   
              None                                 81.55 (±1.29)   

                                         metrics.test/acc_mfpt-2  \
params.epochs params.activation_sparsity                           
100           0.3                                  97.35 (±0.43)   
              0.5                                  96.70 (±0.62)   
              0.7                                  94.97 (±0.78)   
              0.9                                  93.28 (±0.60)   
              None                                 87.64 (±1.46)   
200           0.3                                  97.35 (±1.02)   
              0.5                                  97.83 (±0.51)   
              0.7                                  94.83 (±1.25)   
              0.9                                  93.63 (±0.87)   
              None                                 88.75 (±1.49)   
300           0.3                                  96.39 (±1.06)   
              0.5                                  96.14 (±1.22)   
              0.7                                  94.49 (±1.32)   
              0.9                                  92.32 (±1.02)   
              None                                 89.96 (±1.59)   

                                         metrics.test/acc_mfpt0  \
params.epochs params.activation_sparsity                          
100           0.3                                 99.04 (±0.18)   
              0.5                                 98.82 (±0.23)   
              0.7                                 97.67 (±0.54)   
              0.9                                 95.71 (±0.58)   
              None                                95.71 (±1.10)   
200           0.3                                 99.24 (±0.46)   
              0.5                                 99.33 (±0.20)   
              0.7                                 98.00 (±0.59)   
              0.9                                 97.42 (±0.33)   
              None                                96.36 (±0.84)   
300           0.3                                 98.81 (±0.58)   
              0.5                                 98.60 (±0.93)   
              0.7                                 98.04 (±0.69)   
              0.9                                 97.45 (±0.31)   
              None                                96.67 (±1.11)   

                                         metrics.test/acc_mfpt2  \
params.epochs params.activation_sparsity                          
100           0.3                                 99.58 (±0.12)   
              0.5                                 99.35 (±0.11)   
              0.7                                 98.43 (±0.52)   
              0.9                                 96.52 (±0.50)   
              None                                99.21 (±0.35)   
200           0.3                                 99.67 (±0.23)   
              0.5                                 9

In [31]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'Featuremap Pruning OTTAWA'")
runs = mlflow.search_runs(filter_string="params.model_name = 'ticnn'",experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_ottawa{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_ottawa']

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(30)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_ottawa-4  \
params.epochs params.activation_sparsity                             
100           0.3                                    60.43 (±2.34)   
              0.5                                    59.89 (±2.29)   
              0.7                                    57.54 (±1.82)   
              0.9                                    51.01 (±1.61)   
              None                                   60.28 (±1.96)   
200           0.3                                    63.77 (±1.62)   
              0.5                                    63.34 (±2.38)   
              0.7                                    57.24 (±2.45)   
              0.9                                    51.39 (±1.53)   
              None                                   59.81 (±2.21)   
300           0.3                                    63.81 (±2.89)   
              0.5                                    60.17 (±2.08)   
              0.7                                    57.63 (±2.32)   
              0.9                                    51.17 (±1.96)   
              None                                   60.58 (±2.06)   

                                         metrics.test/acc_ottawa-2  \
params.epochs params.activation_sparsity                             
100           0.3                                    75.02 (±2.11)   
              0.5                                    74.24 (±1.97)   
              0.7                                    71.37 (±1.63)   
              0.9                                    65.72 (±1.52)   
              None                                   73.32 (±2.09)   
200           0.3                                    78.03 (±1.46)   
              0.5                                    77.21 (±1.84)   
              0.7                                    71.68 (±2.25)   
              0.9                                    65.97 (±1.43)   
              None                                   73.01 (±2.33)   
300           0.3                                    77.05 (±2.42)   
              0.5                                    75.45 (±2.18)   
              0.7                                    72.37 (±2.25)   
              0.9                                    64.80 (±1.88)   
              None                                   73.74 (±1.93)   

                                         metrics.test/acc_ottawa0  \
params.epochs params.activation_sparsity                            
100           0.3                                   86.70 (±1.42)   
              0.5                                   86.16 (±1.36)   
              0.7                                   83.46 (±1.29)   
              0.9                                   78.80 (±1.22)   
              None                                  84.62 (±1.72)   
200           0.3                                   88.38 (±1.23)   
              0.5                                   87.37 (±1.52)   
              0.7                                   84.31 (±1.65)   
              0.9                                   79.10 (±1.12)   
              None                                  84.24 (±1.98)   
300           0.3                                   87.14 (±1.71)   
              0.5                                   86.86 (±1.73)   
              0.7                                   84.27 (±1.98)   
              0.9                                   78.02 (±1.49)   
              None                                  85.02 (±1.46)   

                                         metrics.test/acc_ottawa2  \
params.epochs params.activation_sparsity                            
100           0.3                                   93.67 (±0.70)   
              0.5                                   93.58 (±0.70)   
              0.7                                   92.38 (±0.85)   
              0.9                                   88.11 (±0.74)   
              None                                  92.52 (±1.02)   
20

In [74]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'Compound Pruning %'")
runs = mlflow.search_runs(filter_string="params.model_name = 'wdcnn'",experiment_ids=[experiment.experiment_id for experiment in experiments])

cols = [f'metrics.test/acc_cwru{n}' for n in [-4, -2, 0, 2, 4]] + ['metrics.test/acc_cwru']

runs[cols] = runs[cols] * 100

means = runs.groupby(by=['params.epochs', 'params.activation_sparsity', 'params.weight_sparsity'])[cols].mean()
stds = runs.groupby(by=['params.epochs', 'params.activation_sparsity', 'params.weight_sparsity'])[cols].std()
ci_95 = 1.96 * stds / np.sqrt(30)

gen_ci_str = lambda mean, ci: f'{mean:.2f} (±{ci:.2f})'
def generate_CI_str(means:pd.Series, stds:pd.Series):
    return means.combine(stds, gen_ci_str)

result_table = means.combine(ci_95, generate_CI_str)
result_table

metrics.test/acc_cwru-4  \
params.epochs params.activation_sparsity params.weight_sparsity                           
100           0.3                        0.3                              43.17 (±2.69)   
              0.5                        0.5                              42.80 (±2.77)   
200           0.3                        0.3                              45.08 (±3.28)   
              0.5                        0.5                              47.97 (±3.08)   
300           0.3                        0.3                              47.34 (±3.35)   
              0.5                        0.5                              49.87 (±2.95)   

                                                                metrics.test/acc_cwru-2  \
params.epochs params.activation_sparsity params.weight_sparsity                           
100           0.3                        0.3                              61.77 (±2.71)   
              0.5                        0.5                              60.16 (±2.78)   
200           0.3                        0.3                              63.70 (±3.28)   
              0.5                        0.5                              65.11 (±2.83)   
300           0.3                        0.3                              67.01 (±3.24)   
              0.5                        0.5                              68.17 (±2.76)   

                                                                metrics.test/acc_cwru0  \
params.epochs params.activation_sparsity params.weight_sparsity                          
100           0.3                        0.3                             80.43 (±2.03)   
              0.5                        0.5                             76.79 (±2.19)   
200           0.3                        0.3                             81.33 (±2.69)   
              0.5                        0.5                             80.37 (±1.80)   
300           0.3                        0.3                             84.35 (±2.42)   
              0.5                        0.5                             83.29 (±1.98)   

                                                                metrics.test/acc_cwru2  \
params.epochs params.activation_sparsity params.weight_sparsity                          
100           0.3                        0.3                             91.46 (±1.16)   
              0.5                        0.5                             88.27 (±1.27)   
200           0.3                        0.3                             92.45 (±1.73)   
              0.5                        0.5                             90.75 (±1.09)   
300           0.3                        0.3                             94.31 (±1.09)   
              0.5                        0.5                             92.44 (±0.93)   

                                                                metrics.test/acc_cwru4  \
params.epochs params.activation_sparsity params.weight_sparsity                          
100           0.3                        0.3                             96.26 (±0.49)   
              0.5                        0.5                             94.21 (±0.46)   
200           0.3                        0.3                             96.78 (±0.87)   
              0.5                        0.5                             95.23 (±0.52)   
300           0.3                        0.3                             97.71 (±0.30)   
              0.5                        0.5                             96.13 (±0.38)   

                                                                metrics.test/acc_cwru  
params.epochs params.activation_sparsity params.weight_sparsity                        
100           0.3                        0.3                            98.95 (±0.16)  
              0.5                        0.5                            98.30 (±0.22)  
200           0.3                        0.3                            99.01 (±0.19)  
   

In [ ]:
epochs = 300

fig, axes = plt.subplots(1, 2, sharey='row', figsize=(20, 10))

width = 0.35
x = np.arange(4)
labels = ['0.3', '0.5', '0.7', '0.9']
environs = ('cwru', 'cwru0')

runs = mlflow.search_runs(experiment_names=['WDCNN None Pruning'])
runs[['metrics.test/acc_cwru', 'metrics.test/acc_cwru0']] = runs[['metrics.test/acc_cwru', 'metrics.test/acc_cwru0']] * 100
dense_acc = runs.groupby('params.epochs')[['metrics.test/acc_cwru', 'metrics.test/acc_cwru0']].mean()

for environ, ax in zip(environs, axes):
    unstr_bar = ax.bar(x - width/2, round(means.loc[str(epochs), 'activation'][f'metrics.test/acc_{environ}'], 2), width, label='Unstructured', color='C1', linewidth=1.5, edgecolor='black')
    str_bar = ax.bar(x + width/2, round(means.loc[str(epochs), 'channel'][f'metrics.test/acc_{environ}'], 2), width, label='Structured', color='C2', linewidth=1.5, edgecolor='black')

    ax.bar_label(unstr_bar, padding=3, fontsize=18)
    ax.bar_label(str_bar, padding=3, fontsize=18)

    dense = dense_acc.loc[str(epochs)][f'metrics.test/acc_{environ}']

    dense_line = ax.hlines(dense, 0, 1, transform=ax.get_yaxis_transform(), linewidth=3, linestyles='dashed', color='C0')

    if environ == 'cwru':
        ax.set_title('Noise-free', fontsize=28)
    else:
        ax.set_title('Noisy (0dB)', fontsize=28)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.tick_params(axis='both', labelsize=24, which='major', bottom=True, left=True, grid_color='none', color='black', length=8, width=2)

axes[0].set_ylim(30)

ax = fig.add_subplot(111, frameon=False)
ax.set_xlabel('Sparsity', fontsize=28, labelpad=20)
ax.set_ylabel('Accuracy (%)', fontsize=28, labelpad=25)
ax.tick_params(grid_color='none',labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
ax.set_title(f'Epoch : {epochs}', y=1.1, fontsize=32)

legend = fig.legend([unstr_bar, str_bar, dense_line], ('Unstructured', 'Structured', 'Dense'), loc='lower left', bbox_to_anchor=(.99, 0.45), fontsize=24)
legend.get_frame().set_linewidth(2)
legend.get_frame().set_edgecolor('black')
fig.tight_layout()